This notebook contains SQL code for building out the tables and views in the SGCN schema of the DataDistillery instance of GC2.

In [1]:
import requests
from IPython.display import display
from bis2 import gc2

### Core SGCN table

This table essentially mirrors the combined schemas of input data between 2005 and 2015 for SWAP/SGCN lists submitted from states. It is used to house the data produced via code that processes the SGCN repository in ScienceBase.

In [24]:
q_createSGCN = "CREATE TABLE IF NOT EXISTS sgcn.sgcn ( \
    id serial primary key, \
    sgcn_state varchar(150), \
    sgcn_year int, \
    scientificname_submitted varchar(255), \
    commonname_submitted varchar(255), \
    taxonomicgroup_submitted varchar(255), \
    sourceid varchar(255), \
    firstyear boolean, \
    sourcefilename varchar(255), \
    sourcefileurl varchar(255) \
    )"
url_createSGCN = gc2.sqlAPI("DataDistillery","BCB")+"&q="+q_createSGCN
print (requests.get(url_createSGCN).json())

{'_execution_time': 0.023, 'success': True, 'affected_rows': 0}


### National List

This view joins the SGCN table with the Taxa Information Registry where unique SGCN submitted names have been processed against taxonomic authorities and other annotation codes. The "SGCN National List" is assembled for names that have been matched against ITIS or WoRMS, or names which were originally identified as being "correct" in the 2005 SWAP exercise.

The view groups on the "scientificname" property from the TIR, which is produced through the TIR Common Properties script to pull the "best" scientific name for use. It aggregates or sums other properties to produce a synthesized list of those taxa that we consider to be on the national list. It creates individual properties for taxonomic rank, match method, and taxonomic group that are used as filtering parameters when the data are piped from the view into an ElasticSearch index. It also produces a full list of states that submitted names that were matched to the national list names. The view excludes any taxa that were not matched to a taxonomic authority.

In [3]:
q_createNationalListView = "CREATE VIEW sgcn.sgcn_nationallist AS \
    SELECT t.scientificname AS scientificname, \
    array_to_string(array_agg(DISTINCT t.registration->>'scientificname'), ',') AS scientificnames_submitted, \
    (array_agg(t.authorityid))[1] AS taxonomicauthorityid, \
    (array_agg(t.commonname))[1] AS commonname, \
    (array_agg(t.rank))[1] AS taxonomicrank, \
    (array_agg(t.matchmethod))[1] AS matchmethod, \
    (array_agg(t.taxonomicgroup))[1] AS taxonomicgroup, \
    array_to_string(array_agg(DISTINCT CASE WHEN s.sgcn_year=2005 THEN s.sgcn_state END), ',') statelist_2005, \
    array_to_string(array_agg(DISTINCT CASE WHEN s.sgcn_year=2015 THEN s.sgcn_state END), ',') statelist_2015, \
    coalesce(array_length(array_remove(array_agg(DISTINCT CASE WHEN s.sgcn_year=2005 THEN s.sgcn_state END), null), 1), 0) sgcn2005, \
    coalesce(array_length(array_remove(array_agg(DISTINCT CASE WHEN s.sgcn_year=2015 THEN s.sgcn_state END), null), 1), 0) sgcn2015 \
    FROM tir.tir t \
    JOIN sgcn.sgcn s ON \
    s.scientificname_submitted = t.registration->>'scientificname' \
    WHERE t.authorityid != 'Not Matched to Taxonomic Authority' \
    AND t.registration->>'source' = 'SGCN' \
    GROUP BY t.scientificname"
url_createNationalListView = gc2.sqlAPI("DataDistillery","BCB")+"&q="+q_createNationalListView
print (requests.get(url_createNationalListView).json())

{'success': True, '_execution_time': 0.148, 'affected_rows': 0, 'auth_check': {'success': True, 'auth_level': None, 'session': None}}


### SGCN Search

This query produces a view that includes every unique name submitted through the SGCN process whether or not it was matched to a taxonomic authority. It is much the same as the view that provides the SGCN National List except that it includes both matched and nonmatched names and it incorporates the JSON data structures from the Taxa Information Registry that are valuable for search and display purposes. This view and its associated ElasticSearch index give us the "species pages" for the SSGCN application. They provide everything necessary in each record to produce those pages.

In [4]:
q_createSGCNSearch = "CREATE VIEW sgcn.sgcn_search AS \
    SELECT t.scientificname AS scientificname, \
    array_to_string(array_agg(DISTINCT t.registration->>'scientificname'), ',') AS scientificnames_submitted, \
    (array_agg(t.authorityid))[1] AS taxonomicauthorityid, \
    (array_agg(t.commonname))[1] AS commonname, \
    (array_agg(t.rank))[1] AS taxonomicrank, \
    (array_agg(t.matchmethod))[1] AS matchmethod, \
    (array_agg(t.taxonomicgroup))[1] AS taxonomicgroup, \
    (array_agg(t.registration))[1]::jsonb AS registration, \
    (array_agg(t.itis))[1]::jsonb AS itis, \
    (array_agg(t.worms))[1]::jsonb AS worms, \
    (array_agg(t.tess))[1]::jsonb AS tess, \
    (array_agg(t.natureserve))[1]::jsonb AS natureserve, \
    (array_agg(t.sgcn))[1]::jsonb AS sgcn, \
    array_to_string(array_agg(CASE WHEN s.sgcn_year=2005 THEN s.sgcn_state ELSE NULL END), ',') statelist_2005, \
    array_to_string(array_agg(CASE WHEN s.sgcn_year=2015 THEN s.sgcn_state ELSE NULL END), ',') statelist_2015, \
    sum(((s.sgcn_year = 2005))::integer) AS sgcn2005, \
    sum(((s.sgcn_year = 2015))::integer) AS sgcn2015 \
    FROM tir.tir t \
    JOIN sgcn.sgcn s ON \
    s.scientificname_submitted = t.registration->>'scientificname' \
    WHERE t.registration->>'source' = 'SGCN' \
    GROUP BY t.scientificname"
url_createSGCNSearch = gc2.sqlAPI("DataDistillery","BCB")+"&q="+q_createSGCNSearch
print (requests.get(url_createSGCNSearch).json())

{'success': True, 'auth_check': {'success': True, 'session': None, 'auth_level': None}, '_execution_time': 0.193, 'affected_rows': 0}


### State Lists

The state lists view is built by grouping on each state from the SGCN table along with the "best" scientific name from the TIR, such that we end up with a list for each state that includes every unique taxon name they have supplied across all years with other core attributes aggregated. The sgcn2005 and sgcn2015 properties, regardless of the actual numbers in those properties, simply indicate that the taxon was submitted for the associated year. There will be cases where this number is greater than 1 when a given submitted name ended up mapping to the same taxon from an authority.

In [8]:
q_createStateLists = "CREATE VIEW sgcn.sgcn_statelists AS \
    SELECT s.sgcn_state, \
    t.scientificname, \
    array_to_string(array_agg(DISTINCT s.scientificname_submitted), ',') AS scientificnames_submitted, \
    (array_agg(t.authorityid))[1] AS taxonomicauthorityid, \
    (array_agg(t.commonname))[1] AS commonname, \
    (array_agg(t.rank))[1] AS taxonomicrank, \
    (array_agg(t.matchmethod))[1] AS matchmethod, \
    (array_agg(t.taxonomicgroup))[1] AS taxonomicgroup, \
    (array_agg(t.cachedate))[1] AS cachedate, \
    sum(((s.sgcn_year = 2005))::integer) AS sgcn2005, \
    sum(((s.sgcn_year = 2015))::integer) AS sgcn2015 \
    FROM sgcn.sgcn s \
    LEFT JOIN tir.tir t ON \
    t.registration->>'scientificname' = s.scientificname_submitted \
    WHERE t.registration->>'source' = 'SGCN' \
    GROUP BY s.sgcn_state, t.scientificname"
url_createStateListView = gc2.sqlAPI("DataDistillery","BCB")+"&q="+q_createStateLists
print (requests.get(url_createStateListView).json())

{'_execution_time': 0.133, 'affected_rows': 0, 'success': True, 'auth_check': {'auth_level': None, 'success': True, 'session': None}}
